<a href="https://colab.research.google.com/github/kkmjkim/nlp-bible-code-2021/blob/master/15%EC%9E%A5_Text%20summarization/%5B15-2%5D%20%EB%B9%84%EC%A7%80%EB%8F%84%20%ED%95%99%EC%8A%B5%EC%9D%84%20%EC%9D%B4%EC%9A%A9%ED%95%9C%20%EC%9D%8C%EC%8B%9D%20%EB%A6%AC%EB%B7%B0%20%EC%B6%94%EC%B6%9C%20%EC%9A%94%EC%95%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import re
from nltk.tokenize import sent_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gensim.models.keyedvectors as word2vec
import gc
import string
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**Download & unzip "Reviews.csv" [here](https://www.kaggle.com/snap/amazon-fine-food-reviews?select=Reviews.csv) </br>**
1) 본인 구글 드라이브에 올리고 코드 그대로 진행 or </br>
2) 바로 아래 코드를 주석 부분으로 바꾼 후 widget에서 파일 선택

**Download & unzip "glove.twitter.27B.25d.txt" [here](https://github.com/stanfordnlp/GloVe) </br>**
** Twitter (2B tweets, 27B tokens, 1.2M vocab, uncased, 200d vectors, 1.42 GB download)</br>
** 우리가 사용할 파일: "glove.twitter.27B.25d.txt" </br>
1) 본인 구글 드라이브에 올리고 코드 그대로 진행 or </br>
2) loadEmbeddingMatrix()의 5~7번 줄을 아래와 같이 변경 </br>

```
# 드라이브에 업로드 시, (현재 코드)
EMBEDDING_FILE = 'drive/MyDrive/glove.twitter.27B.25d.txt'

# colab에서 업로드 시, 이후에 위젯에서 파일 선택
uploaded_glove = files.upload()
EMBEDDING_FILE = io.BytesIO(uploaded['glove.twitter.27B.25d.txt'])
EMBEDDING_FILE = 'glove.twitter.27B.25d.txt'
```



In [2]:
# 매번 upload 너무 오래 걸려서 따로 Google drive에 업로드 후 여기서 mount
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

import pandas as pd 
df = pd.read_csv('drive/MyDrive/Reviews.csv')

# from google.colab import files
# uploaded = files.upload()
# 
# import io
# df = pd.read_csv(io.BytesIO(uploaded['Reviews.csv']))
# # Dataset is now stored in a Pandas Dataframe (two-dimensional data structure with labeled axes)

Mounted at /content/drive


In [3]:
df.head(3)

Id  ...                                               Text
0   1  ...  I have bought several of the Vitality canned d...
1   2  ...  Product arrived labeled as Jumbo Salted Peanut...
2   3  ...  This is a confection that has been around a fe...

[3 rows x 10 columns]

In [4]:
# Splits the reviews into individual sentences
def split_sentences(reviews):
    n_reviews = len(reviews)
    for i in range(n_reviews):
        review = reviews[i]
        #print(email)
        sentences = sent_tokenize(review)
        #print(sentences)
        for j in reversed(range(len(sentences))): # b/c stack
            sent = sentences[j]
            sentences[j] = sent.strip() # 양쪽 끝 공백, \n 제거 (X 중간)
            if sent == '':
                sentences.pop(j)
        reviews[i] = sentences # # in splitted format

In [5]:
rev_list = list(df['Text'])
split_sentences(rev_list)
#Adding split reviews in the data frame
df['sent_tokens'] = rev_list # new col
#Calculating length of sentences in each review
df['length_of_rv'] = df['sent_tokens'].map(lambda x: len(x)) # new col

In [6]:
# 그냥 df.head()하면 중간 column들 생략됨
pd.set_option('display.max_columns', df.shape[1])
print(df.head())

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  \
0  Good Quality Dog Food  I have bought several of the Vitality canned d...   


In [7]:
choice_length = 5
# select reviews with >5 sentences (b/c tryin to summarize)
df = df[df["length_of_rv"] > choice_length]
df.shape

(172765, 12)

In [8]:
#Making vocabulary with reviews with max vocabs=5000. 
list_sentences_train = df['Text']
max_features = 5000 # out of 393579 unique values

tokenizer = Tokenizer(num_words=max_features) # Tokenizer 객체
tokenizer.fit_on_texts(list(list_sentences_train)) # 빈도수를 기준으로 num_words-1개 단어 인덱스 구축 393579 -> 5000
# print(list_sentences_train[0]) # "I have bought several .."

list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
# print(list_tokenized_train[0:2]) # [[I have bought..],[Product arrived labeled..]] -> [[2, 17, 125, ..], [38, 376, 2210, ..]]

maxlen = 200
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen) # [[0, 0,.., 2, 17, .., 140], [0, 0,.., 38, 376, .., 23], ..]

>loadEmbeddingMatrix()의 5~7줄을 상황에 따라 수정

```
# "glove.twitter.27B.25d.txt"를 드라이브에 업로드 시, (현재 코드)
EMBEDDING_FILE = 'drive/MyDrive/glove.twitter.27B.25d.txt'

# colab에서 업로드 시, 이후에 위젯에서 파일 선택
uploaded_glove = files.upload()
EMBEDDING_FILE = io.BytesIO(uploaded['glove.twitter.27B.25d.txt'])
EMBEDDING_FILE = 'glove.twitter.27B.25d.txt'
```


In [9]:
def loadEmbeddingMatrix(typeToLoad):
        #load different embedding file from Kaggle depending on which embedding 
        #matrix we are going to experiment with
        if typeToLoad=="glove":
            # uploaded_glove = files.upload()
            # EMBEDDING_FILE = io.BytesIO(uploaded['glove.twitter.27B.25d.txt'])
            EMBEDDING_FILE = 'drive/MyDrive/glove.twitter.27B.25d.txt'
            embed_size = 25
        
        elif typeToLoad=="fasttext":
            EMBEDDING_FILE='drive/MyDrive/wiki.simple.vec/wiki.simple.vec'
            embed_size = 300

        if typeToLoad=="glove" or typeToLoad=="fasttext":
            embeddings_index = dict()
            #Transfer the embedding weights into a dictionary by iterating through every line of the file.
            f = open(EMBEDDING_FILE, encoding='utf-8')
            for line in f: # ex) that 0.20823 0.22476 ..... 0.15874 -0.55338
                #split up line into an indexed array
                values = line.split() # 공백 기준으로 split
                #first index is word ex) "that"
                word = values[0]
                #store the rest of the values in the array as a new array
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs # 25 dimensions (우리가 정한 "glove.twitter.27B.25d.txt")
            f.close()
            print('Loaded %s word vectors.' % len(embeddings_index)) # 현재 파일은 1.2M vocab
       # else:
        #    embeddings_index = dict()
         #   for word in word2vecDict.wv.vocab:
          #      embeddings_index[word] = word2vecDict.word_vec(word)
           # print('Loaded %s word vectors.' % len(embeddings_index))
            
        gc.collect()
        return embeddings_index #, embedding_matrix

In [10]:
## Loading 'glove' words
emb_index= loadEmbeddingMatrix('glove') # dictionary

Loaded 1193515 word vectors.


In [11]:
def calculate_sentence_embedding(wordList):
    """
    This function calculates the embedding for entire sentence by taking the mean of embedding of
    each word in the sentence. To be improved. 
    """
    #wordList = re.sub("[^\w]", " ",  sent).split()
    #print(wordList)
    emb_li = []
    for k in wordList: # ['the', 'cat', 'sat', 'on', 'the', 'mat', '0_0']
        embedding_vector = emb_index.get(k) # embedding in GloVe(vector)! ex) the의 벡터: [-0.010167, 0.020194, ..... -0.8425, -0.31208]; np.array임
        if embedding_vector is not None:
            if len(embedding_vector) == 25:
                emb_li.append(list(embedding_vector))
    #print("Lost words in translation: ", len(wordList)-len(emb_li))
    mean_arr = np.array(emb_li)
    #print("done calculating sentence emb for you")
    return np.mean(mean_arr, axis=0) # (, 25) : the vector for this sentence


def get_sent_embedding(mylist):
    """
    This function calculates the embedding of each sentence in the review. Checks if the sentence being passed is a valid one, 
    removing the punctuation and emojis etc.
    """
    sent_emb = []
    n_sentences = len(mylist) # sentences in one review
    for i in mylist:
        #print("my sentence : ", i)
        #print("\nlower is", i.lower())
        i = i.lower()

        # re.sub(): i에서 "[^\w]"가 안 겹치면 가장 왼쪽 일치를 " "로 치환 ex) @cat ->  cat; c@$at -> c at
        # ^: start of line; \w: metacharacter [a-zA-Z0-9_]
        wL = re.sub("[^\w]", " ",  i).split() # "the cat !sat on^the _ mat . 0_0" -> ['the', 'cat', 'sat', 'on', 'the', '_', 'mat', '0_0']
        if len(wL) > 0:
            for k in wL: # 'the'
                if k in string.punctuation: # 단독인 '_'는 remove됨; '0_0'는 안 됨
                    wL.remove(k)
            if len(wL) <= 2:
                continue
        else:
            print("Sentence Removed: ",i) # mostly punctuations / emojis
            continue
        #print(wL)
        res = list(calculate_sentence_embedding(wL))
        sent_emb.append(res)
    return np.array(sent_emb) # (n_sentences, 25)

In [13]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
## Calculating summaries for first 5000 reviews. (out of 568,454)
how_many_summaries = 5000
summary = [None]*how_many_summaries
for rv in range(how_many_summaries):
        review = df['sent_tokens'].iloc[rv]
        enc_email = get_sent_embedding(review)
        # print("enc_email:", enc_email.shape) # (8, 25)
        if len(enc_email) > 0:
            n_clusters = int(np.ceil(len(enc_email)**0.5)) # 9 -> 3 / 10 -> 4 ... 요약으로 생성할 문장 수
            kmeans = KMeans(n_clusters=n_clusters, random_state=0) # k in kmeans
            kmeans = kmeans.fit(enc_email)
            avg = []
            closest = []
            for j in range(n_clusters):
                # np.where(condition) : if only condition, == np.asarray(condition).nonzero()
                idx = np.where(kmeans.labels_ == j)[0] # [1 2 3 7], [4 5], [0 6] 끼리 연관성이 있음
                # print("cluster# and IDX: ", j, idx) # 0 [1 2 3 7]
                avg.append(np.mean(idx))
                
            # for each row in X, which index of the row of Y is closest? returns: Y[argmin[i], :], distances[i]
            closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_,\
                                                       enc_email) # 결국 가장 대표적인 문장을 선택 (생성 X)
            ordering = sorted(range(n_clusters), key=lambda k: avg[k]) # avg: [3.xx, 4.5, 3] -> avg[2], avg[0], avg[1] (기존 내용 순서 고려한 것)
            summary[rv] = ' '.join([review[closest[idx]] for idx in ordering])
            print("Done for review # = ", rv)
            # print("Done for review # = ", rv, summary[rv])
        else:
            print("This is not a valid review")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Done for review # =  389
Done for review # =  390
Done for review # =  391
Done for review # =  392
Done for review # =  393
Done for review # =  394
Done for review # =  395
Done for review # =  396
Done for review # =  397
Done for review # =  398
Done for review # =  399
Done for review # =  400
Done for review # =  401
Done for review # =  402
Done for review # =  403
Done for review # =  404
Sentence Removed:  :-)
Done for review # =  405
Done for review # =  406
Done for review # =  407
Done for review # =  408
Done for review # =  409
Done for review # =  410
Done for review # =  411
Done for review # =  412
Sentence Removed:  !
Done for review # =  413
Done for review # =  414
Done for review # =  415
Done for review # =  416
Done for review # =  417
Done for review # =  418
Done for review # =  419
Done for review # =  420
Done for review # =  421
Done for review # =  422
Sentence Removed:  !
Done for review # =  423
Done for review # =  424

In [14]:
df_5000 = df.iloc[:5000]
df_5000['PredictedSummary'] = summary
df_5000[['Text', 'PredictedSummary']].to_csv('top_5000_summary.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
df_result = pd.read_csv('top_5000_summary.csv')
df_result.head(3)

Unnamed: 0                                               Text  \
0           2  This is a confection that has been around a fe...   
1           7  This taffy is so good.  It is very soft and ch...   
2          30  I have never been a huge coffee fan. However, ...   

                                    PredictedSummary  
0  This is a confection that has been around a fe...  
1  I would definitely recommend you buying it. It...  
2  ).<br />The little Dolche Guesto Machine is su...